In [81]:
import os
from fastai import *
from fastai.vision import *
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from torch import nn
from torch import sigmoid
from torchsummary import summary
import torch
from glob import glob
import cv2 as cv2
from pathlib import Path
import PIL
import numpy as np
from torchvision import models, transforms
from torch.autograd import Variable
import torch.onnx


In [2]:
# load the charachter recognition model
char_rec = load_learner(path=".", file= "dataset/char_pred_model.pkl")

In [3]:
clean_paths = glob("clean_chars/*.jpg")

In [67]:
path = random.sample(clean_paths,1)[0]

### FastAI prediction

In [68]:
char = cv2.imread(str(path),cv2.IMREAD_GRAYSCALE)
char = cv2.resize(char, (29,58), interpolation = cv2.INTER_AREA)
char3d = np.stack((char,)*3, axis=-1).astype('float32')
sample = Image(pil2tensor(char3d, dtype=np.float32).div(255))
sample

In [69]:
char_rec.predict(sample)

(Category tensor(4),
 tensor(4),
 tensor([9.7057e-13, 1.6351e-09, 1.3827e-11, 6.0792e-12, 1.0000e+00, 4.3914e-08,
         1.9133e-08, 1.1222e-11, 7.4791e-13, 4.0915e-10, 1.2934e-11, 2.8521e-12,
         9.7255e-10, 2.0151e-10, 5.1012e-08, 5.7759e-10, 5.3211e-11, 8.4891e-07,
         1.7423e-11, 7.0439e-10, 5.7258e-08, 4.5264e-08, 3.5767e-09, 2.9285e-10,
         1.3861e-12, 1.2902e-10, 2.9175e-08, 7.2409e-12, 1.1474e-12, 3.7425e-12,
         9.8266e-11, 2.6397e-10, 7.1918e-09, 1.3392e-12, 4.0529e-11, 4.1543e-11]))

### Pytorch prediction

In [70]:
state_dict = torch.load(Path('dataset/models/stage-2-ocr.pth'))
#print(state_dict.keys())
char_rec_model = char_rec.model

In [71]:
char_rec_model.load_state_dict(state_dict['model'], strict= False)
char_rec_model.eval();

In [72]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [78]:
trans = transforms.Compose([transforms.ToPILImage(),
    transforms.Resize((58, 29)),transforms.ToTensor(),
                            transforms.Normalize(tuple(imagenet_stats[0]), tuple(imagenet_stats[1]))])
char3d = np.stack((char,)*3, axis=-1)
input_img = trans(char3d)

In [79]:
pred = char_rec_model(input_img.unsqueeze(0).cuda())
pred = pred.data.cpu().numpy()
soft_max_pred = softmax(pred[0])
np.argmax(soft_max_pred), np.max(soft_max_pred)

(4, 0.9999988)

In [80]:
soft_max_pred

array([9.705752e-13, 1.635061e-09, 1.382747e-11, 6.079163e-12, 9.999988e-01, 4.391427e-08, 1.913303e-08, 1.122202e-11,
       7.479117e-13, 4.091464e-10, 1.293377e-11, 2.852052e-12, 9.725494e-10, 2.015127e-10, 5.101236e-08, 5.775943e-10,
       5.321073e-11, 8.489100e-07, 1.742267e-11, 7.043936e-10, 5.725811e-08, 4.526422e-08, 3.576683e-09, 2.928456e-10,
       1.386078e-12, 1.290169e-10, 2.917458e-08, 7.240905e-12, 1.147416e-12, 3.742545e-12, 9.826614e-11, 2.639695e-10,
       7.191805e-09, 1.339172e-12, 4.052938e-11, 4.154345e-11], dtype=float32)

### Export to ONNX

In [83]:
torch.onnx.export(char_rec_model, input_img.unsqueeze(0).cuda(),
                      "char_model.onnx",
                      verbose=True)

graph(%input.1 : Float(1, 3, 58, 29),
      %0.0.weight : Float(64, 3, 7, 7),
      %0.1.weight : Float(64),
      %0.1.bias : Float(64),
      %0.1.running_mean : Float(64),
      %0.1.running_var : Float(64),
      %0.4.0.conv1.weight : Float(64, 64, 1, 1),
      %0.4.0.bn1.weight : Float(64),
      %0.4.0.bn1.bias : Float(64),
      %0.4.0.bn1.running_mean : Float(64),
      %0.4.0.bn1.running_var : Float(64),
      %0.4.0.conv2.weight : Float(64, 64, 3, 3),
      %0.4.0.bn2.weight : Float(64),
      %0.4.0.bn2.bias : Float(64),
      %0.4.0.bn2.running_mean : Float(64),
      %0.4.0.bn2.running_var : Float(64),
      %0.4.0.conv3.weight : Float(256, 64, 1, 1),
      %0.4.0.bn3.weight : Float(256),
      %0.4.0.bn3.bias : Float(256),
      %0.4.0.bn3.running_mean : Float(256),
      %0.4.0.bn3.running_var : Float(256),
      %0.4.0.downsample.0.weight : Float(256, 64, 1, 1),
      %0.4.0.downsample.1.weight : Float(256),
      %0.4.0.downsample.1.bias : Float(256),
      %0.4.0.down

### Convert to pb format for tensorflow

In [ ]:
!onnx-tf convert -i char_model.onnx -o char_model.pb